In [6]:
from z3 import *
from itertools import combinations
import time
import numpy as np


# Aux functions

# start_times = x and y coords
# durations and resources = x and y dims
# total = width / height
def cumulative(start_times, durations, resources, total):
    cumulative_expression = []
    for resource in resources:
        cumulative_expression.append(
            sum([
                If(And(start_times[s] <= resource, resource < start_times[s] + durations[s]), 
                resources[s], 
                0) 
                for s in range(len(start_times))
            ]) <= total
        )
    #print(cumulative_expression)
    return cumulative_expression

def cumulative2(opt, s, d, r, c, time):
    for t in range(time):
        tmp_sum = 0
        for i in range(len(s)):
            tmp_sum += If(And(s[i] <= t, t < s[i] + d[i]), 1, 0) * r[i]
        opt.add(c >= tmp_sum) 
    return opt

def maximum(array):
    maximum = array[0]
    for value in array[1:]:
        maximum = If(value > maximum, value, maximum)
    return maximum

# Instance 
class Instance(object):
    width = 0
    n = 0
    dimensions = []
    def __init__(self, width, n, dimensions):
        self.width = width
        self.n = n
        self.dimensions = dimensions

# Read instances: 
def read_file(file_name):
    dimensions = []
    with open(file_name) as f:
        width = int(f.readline())                 # Width of the plate
        n = int(f.readline())                     # Number of blocks
        while True:
            line = f.readline()
            if not line: 
                break
            dimensions.append(line.split(" "))    # Dimensions of each plate
    for dim in dimensions:
        dim[0] = int(dim[0])
        dim[1] = int(dim[1])
    instance = Instance(width, n, dimensions)
    return instance


# Solve instance: 
def solve(instance):
    optimizer = Optimize()
    
    timeout = 300 * 1000
    optimizer.set("timeout", timeout)

    # Variable Initalization

    x_dims=[]
    y_dims=[]
    x_coords = []
    y_coords = []

    for x_dim,y_dim in instance.dimensions:
        x_dims.append(x_dim)
        y_dims.append(y_dim)
    
    for block in range(instance.n):
        x_coords.append(Int("x%s" % str(block+1)))
        y_coords.append(Int("y%s" % str(block+1)))

    max_height = math.ceil(sum(y_dims)/(instance.width//max(x_dims)))

    print("Width of the plate: ")
    print(instance.width)
    print("Max height of the plate: ")
    print(max_height)

    # Bounding the domain
    
    optimizer.add([x_coords[block] >= 0 for block in range(instance.n)]) # x domain
    optimizer.add([y_coords[block] >= 0 for block in range(instance.n)]) # y domain
        
    optimizer.add([maximum([x_coords[block] + x_dims[block] for block in range(instance.n)]) <= instance.width]) # fits width
    optimizer.add([maximum([y_coords[block] + y_dims[block] for block in range(instance.n)]) <= max_height]) # fits height
    
    # All Different and Cummulative constraints
    
    optimizer.add([Distinct([1000*x_coords[block]+y_coords[block]]) for block in range(instance.n)])
    #optimizer.add(cumulative(y_coords,y_dims,x_dims,instance.width))
    #optimizer.add(cumulative(x_coords,x_dims,y_dims,max_height))
    optimizer = cumulative2(optimizer, x_coords, x_dims, y_dims, max_height, instance.width)
    optimizer = cumulative2(optimizer, y_coords, y_dims, x_dims, instance.width, max_height)
    
    # No overlapping
    
    for (block1,block2) in combinations(range(instance.n),2):
        optimizer.add(Or(x_coords[block1]+x_dims[block1] <= x_coords[block2],
                        x_coords[block2]+x_dims[block2] <= x_coords[block1],
                        y_coords[block1]+y_dims[block1] <= y_coords[block2],
                        y_coords[block2]+y_dims[block2] <= y_coords[block1]))
    
    
    # Goal: minimize height
    
    real_height = maximum([y_coords[block] + y_dims[block] for block in range(instance.n)])
    
    optimizer.minimize(real_height)

   
    
    done = True
    if optimizer.check() == sat:
        model = optimizer.model()
        solution_height = model.evaluate(real_height).as_string()
        print("Satisfiable")
        print("Solution - Minimum height:")
        print(solution_height)
    else:
        print("Not satisfiable")
        done = False
        
    return optimizer, done

def main():
#    instance_file = "../instances/ins-12.txt"
#    instance = read_file(instance_file)

#    start = time.time()
#    optimizer, done = solve(instance)
#    end = time.time()

#    print("{:.2f}".format(end - start) + " seconds")
    
    #IF YOU ONLY WANT TO RUN 12 look at the bottom code
    
    #for x in [19]:
    for x in range(30,39):
        
        instance_file = "../../instances/ins-{}.txt".format(x)
        instance = read_file(instance_file)
        
        start = time.time()
        optimizer, done = solve(instance)
        end = time.time()
        
        print("{:.2f}".format(end - start) + " seconds")
        
        if done:
            with open('ins{}SMT.txt'.format(x), 'w') as myfile:
                myfile.write('time='+'{:.3f}'.format(end - start))
                myfile.close()
        else:
            with open('ins{}SMT.txt'.format(x), 'w') as myfile:
                myfile.write("time=0.000")
                myfile.close()
        
    



if __name__ == '__main__':
    main()

Width of the plate: 
37
Max height of the plate: 
99
Not satisfiable
545.75 seconds
Width of the plate: 
38
Max height of the plate: 
190
Satisfiable
Solution - Minimum height:
38
141.45 seconds
Width of the plate: 
39
Max height of the plate: 
137
Not satisfiable
871.73 seconds
Width of the plate: 
40
Max height of the plate: 
240
Satisfiable
Solution - Minimum height:
40
46.34 seconds
Width of the plate: 
15
Max height of the plate: 
66
Not satisfiable
307.83 seconds
Width of the plate: 
15
Max height of the plate: 
90
Not satisfiable
419.74 seconds
Width of the plate: 
15
Max height of the plate: 
95
Not satisfiable
677.92 seconds
Width of the plate: 
30
Max height of the plate: 
155
Not satisfiable
481.47 seconds
Width of the plate: 
30
Max height of the plate: 
145
Not satisfiable
1624.37 seconds


In [3]:
stat = "time="

times = []

for x in range(1,40):
    f = open('ins{}SMT.txt'.format(x),'r')
    txt = f.read()
    start = txt.find(stat)
    newtxt = txt[start:]
    end = newtxt.find('\n')
    print(str(x)+",",newtxt[len(stat):end])
    times.append(float(newtxt[len(stat):end]))


1, 0.32
2, 0.13
3, 0.23
4, 0.43
5, 0.51
6, 1.36
7, 0.79
8, 0.88
9, 1.05
10, 1.88
11, 69.50
12, 2.62
13, 2.05
14, 2.10
15, 4.07
16, 0.00
17, 7.10
18, 4.42
19, 0.00
20, 53.44
21, 0.00
22, 0.00
23, 60.72
24, 14.81
25, 0.00
26, 129.86
27, 13.64
28, 92.91
29, 576.80
30, 0.00
31, 141.44
32, 0.00
33, 46.33
34, 0.00
35, 0.00
36, 0.00
37, 0.00
38, 0.00


FileNotFoundError: [Errno 2] No such file or directory: 'ins39SMT.txt'